# Multi-Layer Perceptron with MNIST

* [Open in github](https://github.com/cybertraining-dsc/sp21-599-358/blob/main/Assignments/Assignment4/JesusBadillo-rnn-mlp-mnist.ipynb)
* [Open in collab](https://colab.research.google.com/github/cybertraining-dsc/sp21-599-358/blob/main/Assignments/Assignment4/JesusBadillo-rnn-mlp-mnist.ipynb)

## Pre-requisites

Install the following Python packages

1. cloudmesh-installer
2. cloudmesh-common

In [1]:
! pip3 install cloudmesh-installer
! pip3 install cloudmesh-common

     |████████████████████████████████| 81kB 3.6MB/s 
     |████████████████████████████████| 81kB 5.5MB/s 
     |████████████████████████████████| 870kB 6.5MB/s 
     |████████████████████████████████| 133kB 12.3MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
  Created wheel for python-hostlist: filename=python_hostlist-1.21-cp37-none-any.whl size=38931 sha256=2a44c3c3543fe9293a5bd268bb1677698ac53918e2f5aa2aad1cc51cf5ddb875
  Stored in directory: /root/.cache/pip/wheels/0b/5b/55/ddcf52288f0b10f4564ca1b2531594ff7ccc65f487ba8dc437
  Created wheel for ordered-set: filename=ordered_set-4.0.2-py2.py3-none-any.whl size=8210 sha256=2dfade2da589c22811ac638396de844d3f63031bec557709eaefae232cea9a96
  Stored in directory: /root/.cache/pip/wheels/e1/c6/9b/651d8a21d59b51a75ab9c070838f9231b8126421bc0569af47
Successfully built python-hostlist ordered-set
ERROR: pytest-cov 2.11.1 has requirement coverage>=5.2.1, but you'll have coverag

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time 

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D, Flatten, AveragePooling2D, SimpleRNN
from keras.utils import to_categorical, plot_model
from keras.datasets import mnist
#import pydotplus
from keras.utils.vis_utils import model_to_dot
#from keras.utils.vis_utils import pydot

from cloudmesh.common.StopWatch import StopWatch

## Sample MLP with Tensorflow Keras

In [4]:
StopWatch.start("data-load")
(x_train, y_train), (x_test, y_test) = mnist.load_data()
StopWatch.stop("data-load")

num_labels = len(np.unique(y_train))


y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


image_size = x_train.shape[1]
input_size = image_size * image_size


x_train = np.reshape(x_train, [-1, image_size, image_size])
x_train = x_train.astype('float32') / 255
x_test = np.reshape(x_test, [-1, image_size, image_size])
x_test = x_test.astype('float32') / 255

batch_size = 128
hidden_units = 256
units = 256
dropout = 0.45
input_shape = (image_size,image_size)
model = Sequential()

#RNN Layers
model.add(SimpleRNN(units=units,
                    dropout=dropout,
                    input_shape=input_shape, return_sequences=True))
model.add(SimpleRNN(units=units,
                    dropout=dropout,
                    return_sequences=True))
model.add(SimpleRNN(units=units,
                    dropout=dropout,
                    return_sequences=False))
#MLP Layers
model.add(Dense(hidden_units, input_dim=input_size))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(hidden_units))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(hidden_units))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(hidden_units))
model.add(Activation('relu'))
model.add(Dropout(dropout))


#SoftMax Layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

model.summary()
plot_model(model, to_file='mlp-mnist.png', show_shapes=True)


StopWatch.start("compile")
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
StopWatch.stop("compile")
StopWatch.start("train")
model.fit(x_train, y_train, epochs=5, batch_size=batch_size)
StopWatch.stop("train")

StopWatch.start("test")
loss, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))
StopWatch.stop("test")

StopWatch.benchmark()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 28, 256)           72960     
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 28, 256)           131328    
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 256)               131328    
_________________________________________________________________
dense (Dense)                (None, 256)               65792     
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)              

# REFERENCES

1. [Adavnced Keras Deep Learning Book](https://github.com/PacktPublishing/Advanced-Deep-Learning-with-Keras)